<a href="https://colab.research.google.com/github/sangjinsu/keyword-ai/blob/main/keyword.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence_transformers
!pip install konlpy

In [ ]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [ ]:
doc = """
황금올리브치킨 BBQ
by GeniusJW 2018. 10. 12. 21:46



안녕하세요 GeniusJW 입니다. 오늘 소개 해 드릴 메뉴는, BBQ 황금올리브 치킨 입니다. 황금올리브치킨은 BBQ 에서 제가 종종 주문 해 먹는 치킨입니다. 후라이드치킨 중에는 기름도 가장 깨끗한 편이고, 맛있어서 제가 즐겨먹는 메뉴인데요,



황금올리브치킨은 특별한 건 없고, 단지 튀김옷이 바삭하면서 얇은 것이 특징입니다. 다른 치킨가게는 튀김옷이 생각보다 두꺼운 편이고 기름을 많이 머금고 있어서 느끼한 편인데 반면 BBQ 황금올리브 치킨은 생각보단 느끼하지 않고, 올리브오일 향이 은은하게 배어나오는 게 괜찮더라구요.



BBQ 치킨 외에도 후라이드 치킨은 대부분 맛 있는 편이지만, 프랜차이즈 치킨 가운데서는 가장 준수한 맛인 것 같습니다. 써프라이드나 다른 메뉴는 솔직히 개인적으로 만족스럽진 않지만, 황금올리브 치킨 만큼은 인정하는 맛입니다.



오랜만에 황금올리브 치킨을 주문하여 먹었는데요, 맛도 괜찮고 가격도 괜찮아서 앞으로 당분간 황금올리브 치킨 가격이 오르지 않는 이상 종종 주문 해 먹을 것 같습니다. 요즘은 9시 넘어서 KFC 치킨을 먹으면 1+1 이라 주로 KFC 치킨을 먹었는데, KFC는 튀김옷도 약간 두꺼운 편이고, 기름이랑 짠 맛도 좀 강해서 BBQ를 개인적으로 더 선호하는데요, 아무래도 가성비인 KFC도 포기하진 못할 것 같습니다..ㅋㅋㅋ(BBQ도 좋고 KFC도 좋고)
"""
okt = Okt()

tokenized_doc = okt.pos(doc)

tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Adjective' and word[0][-1] not in ['다', '요']])

print('품사 태깅 10개만 출력 :',tokenized_doc[:10])
print('명사 추출 :',tokenized_nouns)


품사 태깅 10개만 출력 : [('\n', 'Foreign'), ('황금', 'Noun'), ('올리브', 'Noun'), ('치킨', 'Noun'), ('BBQ', 'Alpha'), ('\n', 'Foreign'), ('by', 'Alpha'), ('GeniusJW', 'Alpha'), ('2018', 'Number'), ('.', 'Punctuation')]
명사 추출 : 깨끗한 맛있어서 특별한 없고 얇은 두꺼운 있어서 느끼한 느끼하지 은은하게 있는 솔직히 만족스럽진 괜찮고 괜찮아서 두꺼운 강해서 좋고 좋고


In [ ]:
n_gram_range = (1, 1)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :',len(candidates))
print('trigram 다섯개만 출력 :',candidates[:5])

trigram 개수 : 17
trigram 다섯개만 출력 : ['강해서' '괜찮고' '괜찮아서' '깨끗한' '느끼하지']


In [ ]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

In [ ]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)
keywords = [candidates[index] for index in distances.argsort()[0][-top_n:]]
print(keywords)

['괜찮아서', '깨끗한', '만족스럽진', '은은하게', '맛있어서']


In [ ]:
def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates):
    # 문서와 각 키워드들 간의 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)

    # 각 키워드들 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings, 
                                            candidate_embeddings)

    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf
    candidate = None
    for combination in itertools.combinations(range(len(words_idx)), top_n):
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim:
            candidate = combination
            min_sim = sim

    return [words_vals[idx] for idx in candidate]

In [ ]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['얇은', '느끼하지', '강해서', '깨끗한', '은은하게']

In [ ]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=30)

['없고', '있는', '두꺼운', '느끼하지', '은은하게']

In [ ]:
def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity):

    # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

    # 각 키워드들 간의 유사도
    word_similarity = cosine_similarity(candidate_embeddings)

    # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]

    # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]

    # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
    # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
    for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)

    return [words[idx] for idx in keywords_idx]

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.2)

['맛있어서', '은은하게', '깨끗한', '만족스럽진', '두꺼운']

In [ ]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n=5, diversity=0.7)

['맛있어서', '없고', '은은하게', '두꺼운', '얇은']